In [ ]:
!pip install pyjanitor
!pip install kaggle

     |████████████████████████████████| 135 kB 5.1 MB/s 


In [ ]:
import pandas as pd
import janitor

In [ ]:
# Downloading the Lexicon Data
!curl https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip >> NRC-Emotion-Lexicon.zip
!unzip NRC-Emotion-Lexicon.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.3M  100 23.3M    0     0  28.2M      0 --:--:-- --:--:-- --:--:-- 28.2M
Archive:  NRC-Emotion-Lexicon.zip
   creating: NRC-Emotion-Lexicon/
  inflating: __MACOSX/._NRC-Emotion-Lexicon  
  inflating: NRC-Emotion-Lexicon/EmoLex-Ethics-Data-Statement.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._EmoLex-Ethics-Data-Statement.pdf  
  inflating: NRC-Emotion-Lexicon/.DS_Store  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._.DS_Store  
   creating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-v0.92  
  inflating: NRC-Emotion-Lexicon/NRC - Sentiment Lexicon - Research EULA Sept 2017 .pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC - Sentiment Lexicon - Research EULA Sept 2017 .pdf  
  inflating: NRC-Emotion-Lexicon/README.txt  
  inflating: __MACOSX/NRC-Emotio

In [ ]:
# Reading data into panda df
lexicon = pd.read_excel("NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx")
print(lexicon)

      English (en)        Afrikaans (af)     Albanian (sq) Amharic (am)  \
0            aback  uit die veld geslaan             prapa         ተጭኗል   
1           abacus                abakus         numërator       abacus   
2          abandon               verlaat           braktis           ውጣ   
3        abandoned               verlate         braktisur         ተትቷል   
4      abandonment             verlating         braktisje         ማቋረጥ   
...            ...                   ...               ...          ...   
14177         zone                  sone              zonë           ዞን   
14178          zoo             dieretuin  kopsht zoologjik         መናፈሻ   
14179   zoological           dierkundige         zoologjik       ዞኦሎጂካል   
14180      zoology             dierkunde          zoologji         ዞኦሎጂ   
14181         zoom               Klik op              zoom          አጉላ   

       Arabic (ar)      Armenian (hy) Azeerbaijani (az)     Basque (eu)  \
0       الى الوراء      

In [ ]:
# Exploration
for i in lexicon.columns:
  if "English" in i:
    print(i)
# Note that there are two English columns! We'll drop the second
lexicon = lexicon.drop(['English (en).1'], axis=1)

English (en)
English (en).1


In [ ]:
# Cleaning column names
lexicon.rename(columns=lambda x: x.split(" ")[0], inplace=True)
lexicon = lexicon.clean_names()
print(lexicon)

           english             afrikaans          albanian amharic  \
0            aback  uit die veld geslaan             prapa    ተጭኗል   
1           abacus                abakus         numërator  abacus   
2          abandon               verlaat           braktis      ውጣ   
3        abandoned               verlate         braktisur    ተትቷል   
4      abandonment             verlating         braktisje    ማቋረጥ   
...            ...                   ...               ...     ...   
14177         zone                  sone              zonë      ዞን   
14178          zoo             dieretuin  kopsht zoologjik    መናፈሻ   
14179   zoological           dierkundige         zoologjik  ዞኦሎጂካል   
14180      zoology             dierkunde          zoologji    ዞኦሎጂ   
14181         zoom               Klik op              zoom     አጉላ   

            arabic           armenian azeerbaijani          basque  \
0       الى الوراء             շեղում        sanki           aback   
1        طبلية تاج 

In [ ]:
# Creating Emotions DF from subset of Lexicon
emotions = lexicon.iloc[:, -10:]
emotions['word'] = lexicon.loc[:, "english"]
print(emotions.columns)


Index(['positive', 'negative', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'sadness', 'surprise', 'trust', 'word'],
      dtype='object')


In [ ]:
emotions[emotions['word'] == 'sad', :]

TypeError: ignored

In [ ]:
# How many words are positive, negative, angry, etc.? 
print("Words in Lexicon:", len(emotions))
for emotion in emotions:
  if emotion != "word":
    print(emotion + ": ", end="")
    print(emotions.groupby(emotion).size()[1])

In [ ]:
emotions

In [ ]:
# Saving emotions dataset as csv
emotions.to_csv("emotions.csv")

In [ ]:
# Creating translation DF from subset of Lexicon
translation = lexicon.iloc[:, :-10]
print(translation.columns)
translation = translation.dropna()
translation.isnull().values.any()

translation.to_csv("translation.csv")

Index(['english', 'afrikaans', 'albanian', 'amharic', 'arabic', 'armenian',
       'azeerbaijani', 'basque', 'belarusian', 'bengali',
       ...
       'turkish', 'ukrainian', 'urdu', 'uzbek', 'vietnamese', 'welsh', 'xhosa',
       'yiddish', 'yoruba', 'zulu'],
      dtype='object', length=104)


In [ ]:
for c in translation.columns:
  print(f"\"{c.capitalize()}\",")

In [ ]:
!pip install ipdb

In [ ]:
# Pulling synonym dataset from kaggle; will need a kaggle.json api key
!mv kaggle.json /root/.kaggle/
!kaggle datasets download duketemon/wordnet-synonyms
!unzip wordnet-synonyms.zip

In [ ]:
syn = pd.read_csv("synonyms.csv")
syn

In [ ]:
# Number of each part of speech
print(syn.groupby(['part_of_speech']).size())
syn['synonyms'].isnull().sum()

The dataset is pretty messy; a few things need to be cleaned up. 

1.   First, the synonyms are bunched up into one row, separated by semicolons. We want each row to be one word and one synonym, not one word to many synonyms. 
2.   Second, the word itself often appears in the synonyms column. However sometimes it doesn't and a word doesn't have any synonyms at all! We have to drop these rows too. 
3. Lastly, sometimes the words are split on the pipe symbol '|' rather than semicolons.






In [ ]:
import re
%pdb off
# def runner():

syn.dropna(inplace=True)
# counter = 0
synonyms = pd.DataFrame(columns = syn.columns) 
def spl(series):
  # nonlocal counter
  syns = re.split(';|\|',series['synonyms'])
  # counter += 1
  # if counter % 1000 == 0:
    # print(counter)
  for s in syns:
    # synonyms.append({'word': series['lemma'], 'part_of_speech': series['part_of_speech'], 'synonym': s}, ignore_index=True)
    synonyms.loc[len(synonyms.index)] = [series['lemma'], series['part_of_speech'], s]
    # print(synonyms)

syn.apply(spl, axis=1)
print(synonyms)
  # synonyms = pd.DataFrame(columns = ['word', 'synonym'])
  # spl()
  # syn["synonyms"].apply(lambda x: re.split(';|\|',x))
  # syn
  # type(syn['synonyms'])
  # print(syn.dtypes)
  # syn.groupby(['part_of_speech']).size()
# runner()

In [ ]:
synonyms.to_pickle("./syn.pkl")  